<a href="https://colab.research.google.com/github/bonitr02/datasci_7_geospatial/blob/main/datasci_7_geospatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HHA 507 Assignment 7

# Load Packages

In [7]:
import requests
import requests
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import urllib.parse
import os
import json

# Geospatial Data Sets

## Geospatial Data Set #1

In [38]:
df1 = gpd.read_file('/content/Louisville_Metro_KY_-_Firearm_data_intersections_January_1st%2C_2010-February_22nd%2C_2017.geojson')
df1.columns

Index(['ObjectId', 'incident_number', 'ucr_category', 'type_of_firearm',
       'firearm_manufacturer', 'firearm_model', 'firearm_caliber',
       'recovery_date', 'address_geocode_type', 'recovery_block_address',
       'city', 'state', 'recovery_zipcode', 'person_recovered_from_race',
       'person_recovered_from_sex', 'person_recovered_from_age',
       'address_concat', 'x', 'y', 'longitude', 'latitude', 'confidence',
       'source', 'geometry'],
      dtype='object')

## Geospatial Data Set #2

In [39]:
df2 = gpd.read_file('/content/Louisville_Metro_KY_-_Development_Applications_2010-2018_(Historical).geojson')
df2.columns

Index(['ObjectId', 'APNO', 'MixedUse', 'SqFootage', 'ProjType', 'Rooms',
       'Units', 'BldLots', 'NonBldLots', 'AppType', 'WorkType', 'SubWrkType',
       'ProjName', 'Category', 'BOZA', 'BOZADate', 'PC', 'PCDate', 'LDT',
       'LDTDate', 'DRC', 'DRCDate', 'STNO', 'PreDir', 'Address', 'Suffix',
       'PostDir', 'SubD', 'CreYear', 'LastYear', 'XCoord', 'YCoord',
       'Latitude', 'Longitude', 'geometry'],
      dtype='object')

## Geospatial Data Set #3

In [40]:
df3 = gpd.read_file('/content/Louisville_Metro_KY_-_Environmental_Health_Bulk_Data_-_Establishments.geojson')
df3.columns

Index(['EstablishmentID', 'CountyID', 'RCode', 'RCodeDesc', 'EstType',
       'PremiseName', 'PermiseStreetNo', 'PremiseStreet',
       'PremiseExtraAddrLine', 'PremiseCity', 'PremiseState', 'PremiseZip',
       'InspectionIntervalDays', 'IsPayFee', 'IsStateOwned', 'Status',
       'WaterProvider', 'SewageProvider', 'IsBonded', 'RenewalStatus',
       'PermitPrinted', 'Language', 'RiskType', 'MenuType', 'IsCaterer',
       'IsDriveThru', 'IsTrucksOnly', 'Area', 'MailAppIDDesc',
       'MailPermitIDDesc', 'HoursOfOperation', 'DaysOfOperation',
       'MonthsOfOperation', 'opening_date', 'renewal_sent_date',
       'renewal_received_date', 'permit_print_date', 'permit_expiration_date',
       'next_inspection_date', 'bond_expiration_date', 'Quantity1',
       'Quantity1Unit', 'Quantity2', 'Quantity2Units', 'latitude', 'longitude',
       'comments', 'TotalFeeAmt', 'PermitFeeAmt', 'InspectionFeeAmt',
       'VarianceGranted', 'FollowUpDate', 'ObjectId', 'geometry'],
      dtype='object')

## Geospatial Data Set #4

In [43]:
df4 = gpd.read_file('/content/Louisville_Metro_KY_-_Right_of_Way_Permits_(Historical).geojson')
df4.columns

Index(['ObjectId', 'ID', 'APNO', 'FULLNAME', 'ISCONTRACTOR', 'APDTTM', 'APBY',
       'ISSDTTM', 'ISSBY', 'FROM_DATE', 'TO_DATE', 'COOBY', 'COMMENTS_SEARCH',
       'WORKTYPE', 'WORKTYPE_DESCRIPTION', 'PROCESS_STATE', 'LOCATION',
       'STREET_ADDRESS', 'CITY', 'STATE', 'ZIP', 'GPSX', 'GPSY', 'Latitude',
       'Longitude', 'geometry'],
      dtype='object')

## Geospatial Data Set #5



In [44]:
df5 = gpd.read_file('/content/Tree_Inventory.geojson')
df5.columns

Index(['OBJECTID', 'Tree_ID', 'Species_Name', 'DBH__in_',
       'Replacement_Value____', 'Carbon_Storage__lb_', 'Carbon_Storage____',
       'Gross_Carbon_Sequestration__lb_', 'Gross_Carbon_Sequestration____y',
       'Avoided_Runoff__ftÂ__yr_', 'Avoided_Runoff____yr_',
       'Carbon_Avoided__lb_yr_', 'Carbon_Avoided____yr_',
       'Pollution_Removal__oz_yr_', 'Pollution_Removal____yr_',
       'Energy_Savings____yr_', 'Total_Annual_Benefits____yr_', 'xCoordinate',
       'yCoordinate', 'User_ID', 'Date', 'geometry'],
      dtype='object')

In [ ]:
df5[['']]

In [ ]:
df_gpd[['All_Cancer', 'ZIP']].sort_values(by=['All_Cancer'], ascending=False)

In [ ]:
df_gpd['All_Cancer'] = (df_gpd['All_Cancer'] / 1000)

In [ ]:
df_gpd.plot("Breast_Can", legend=True)